In [6]:
from flatfindr.facebook import Facebook
from collections import defaultdict
from time import sleep
import random
from PIL import Image
import requests
from io import BytesIO
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fb = Facebook()
fb.log_in()
item_links = fb.scrape_item_links()


/Users/aduverger/code/aduverger/flatfindr/raw_data


In [8]:
index = random.randint(0, len(item_links) - 1)
url = item_links[index]
db = defaultdict(list)
db['url'].append(url)

fb.driver.get(url)
# Click on 'see more' button to have the full description
buttons = fb.driver.find_elements_by_xpath("//div[@role='button']")
_ = [button.click() for button in buttons if button.text == 'Voir plus']
sleep(random.uniform(0.2, 0.5))

details = fb.driver.find_elements_by_xpath("//span[@dir]")
price, adress, surface, bedrooms, description = [True] * 5
for i in range(len(details)):
    detail = details[i].text
    if price and "$ / mois" in detail:
        db["price($/month)"].append(int(detail[:5].replace(" ", "")))
        price = False
    elif adress and "Montréal, QC" in detail:
        db["adress"].append(detail.replace("Montréal, QC", ""))
        adress = False
    elif surface and "mètres carrés" in detail:
        db["surface(m2)"].append(int(detail[:4].replace(" ", "")))
        surface = False
    elif surface and "pieds carrés" in detail:
        db["surface(m2)"].append(
            round(int(detail[:4].replace(" ", "")) / 10.764)
        )
        surface = False
    elif bedrooms and "chambres · " in detail:
        db["bedrooms"].append(int(detail[0]))
        bedrooms = False
    elif detail in ('Meublé', 'Non meublé'):
        db["furnished"].append(detail)
    elif description and detail == "Description":
        db["description"].append(details[i + 1].text.replace("Voir moins", ""))
        description = False
        
img = fb.driver.find_element_by_xpath("//img[@referrerpolicy='origin-when-cross-origin']")
db["image"].append(img.get_attribute("src"))

for key, value in db.items():
    if key != 'image':
        print(key+':', value[-1])
    else:
        response = requests.get(db['image'][-1])
        display(Image.open(BytesIO(response.content)).resize((400, 400)))
        

url: https://www.facebook.com/marketplace/item/459231015857147/
price($/month): 1300
adress: 
bedrooms: 2
furnished: Non meublé
description: Suite à ma situation de santé , je cède mon bail et je vends mon mobilier et accessoires de coiffure et manucure pose d'ongles résine et poudre à qui veut bien au complet pour se partir en affaires! Je demande 10 000$ pour tout. Veuillez me contacter en privé pour les infos! Super abordable et en plein centre du plateau Mont royal au centre-ville de Montréal !♡♡♡ un bijou pour qui rêve d'avoir son propre salon!♡ vente rapide! Produits, coloration, meubles, laveuse-sécheuse, rangements, etc. 

**Possibilité aussi d'habiter en haut du local! **
J'adore la Propriétaire alors je veux vraiment quelqu'un de bien pour le salon! Possibilité aussi de faire un salon d'esthétique. 
